In [1]:
import warnings
warnings.filterwarnings('ignore')

import math 
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import numpy as np

import matplotlib.pyplot as plt


from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
#import dask_ml.joblib
from joblib import parallel_backend

from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm
from sklearn.preprocessing import StandardScaler

# For scoring
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [2]:
df = pd.read_csv('../data/train_sample_processed.csv', low_memory=False)

In [3]:
# Copied from Data Preprocessing Don't need all these, but for consistency and time let's encode all
cols_to_encode = ['Census_OSArchitecture',
 'AppVersion',
 'Census_DeviceFamily',
 'OsPlatformSubRelease',
 'OsVer',
 'Census_OSEdition',
 'OsBuildLab',
 'Census_MDC2FormFactor',
 'Census_OSSkuName',
 'Census_OSWUAutoUpdateOptionsName',
 'Census_ChassisTypeName',
 'Census_GenuineStateName',
 'Census_ProcessorClass',
 'Census_FlightRing',
 'Census_PowerPlatformRoleName',
 'Processor',
 'PuaMode',
 'SmartScreen',
 'Census_ActivationChannel',
 'Census_OSVersion',
 'EngineVersion',
 'Census_OSInstallTypeName',
 'ProductName',
 'Platform',
 'SkuEdition',
 'Census_PrimaryDiskTypeName',
 'Census_InternalBatteryType',
 'Census_OSBranch',
 'AvSigVersion']

In [4]:
le = LabelEncoder()

for column in cols_to_encode:
    
    df[column] = le.fit_transform(list(df[column]))

In [5]:
cols = ['AvSigVersion',
 'SmartScreen',
 #'DefaultBrowsersIdentifier',
 'CityIdentifier',
 'Census_SystemVolumeTotalCapacity',#3
 'Census_FirmwareVersionIdentifier',
 'Census_OEMModelIdentifier',
 'Census_ProcessorModelIdentifier',
 'Census_IsWIMBootEnabled',
 'CountryIdentifier',
 'OrganizationIdentifier',
 'GeoNameIdentifier',
 'Census_InternalPrimaryDiagonalDisplaySizeInInches',
 'LocaleEnglishNameIdentifier',
 'Census_OSBuildRevision',
 'Census_OSVersion',
 'AVProductStatesIdentifier',
 'Census_OEMNameIdentifier',
 'Wdft_RegionIdentifier',
 'Census_PrimaryDiskTotalCapacity', #18
 'Census_FirmwareManufacturerIdentifier',
 'Census_OSInstallLanguageIdentifier',
 'Census_OSUILocaleIdentifier',
 'Census_OSInstallTypeName',
 'AppVersion',
 'Census_TotalPhysicalRAM',#24
 'AVProductsInstalled',
 'Census_InternalPrimaryDisplayResolutionHorizontal',
 'EngineVersion',
 'Census_InternalBatteryNumberOfCharges',#28
 'OsBuildLab',
 'IeVerIdentifier',
 'Census_InternalPrimaryDisplayResolutionVertical',
 'Census_ChassisTypeName',
 'Census_OSWUAutoUpdateOptionsName',
 'Census_ProcessorCoreCount',
 'Census_ActivationChannel',
 'Census_OSEdition',
 'Census_InternalBatteryType',
 'Census_OSSkuName',
 'HasDetections']

In [15]:
df = df[cols]

scale_cols = [3, 18, 24, 28]

In [7]:
df.fillna('0', inplace=True)

In [8]:
x_cols = list(df.columns)[1:40]

In [9]:
X = np.array(df.iloc[:, 1:40])
y = np.array(df['HasDetections'])

In [10]:
#Need to pass specific columns
sc = StandardScaler()

X[scale_cols] = sc.fit_transform(X[scale_cols])

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [12]:
clf = GradientBoostingClassifier(learning_rate=0.15)
clf.fit(X_train, Y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.15, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [13]:
roc_auc = roc_auc_score(Y_test, clf.decision_function(X_test))
score = clf.score(X_test, Y_test)

print("roc_auc: {}\nscore: {}".format(round(roc_auc, 2), round(score, 2))) #92%

# Let's look at a confusion matrix to see if there is a bias
tn, fp, fn, tp = confusion_matrix(Y_test, clf.predict(X_test)).ravel()
precision = tp/float(tp+fp)
recall = tp/float(tp+fn)

roc_auc: 0.7
score: 0.64


In [14]:
print("Precision is: {}. Recall is: {}".format(round(precision, 2), round(recall, 2)))

Precision is: 0.64. Recall is: 0.64


In [ ]:
math.r